In [1]:


%%capture
%pip install accelerate peft bitsandbytes transformers trl



## Setup

In [2]:
pip install yachalk

Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install seaborn

Note: you may need to restart the kernel to use updated packages.


In [4]:
pip install pyvis

Note: you may need to restart the kernel to use updated packages.


In [5]:
! pip install "unstructured[pdf]"

In [6]:
! pip install --upgrade langchain

In [7]:
! pip install openai==0.28

In [8]:
! mkdir -p ./corpus_input
! mkdir -p ./corpus_output

In [9]:
import pandas as pd
import numpy as np
import os
from langchain.document_loaders import PyPDFLoader, UnstructuredPDFLoader, PyPDFium2Loader
from langchain.document_loaders import PyPDFDirectoryLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from pathlib import Path
import random

## Input data directory
##### data_dir = "cureus"
inputdirectory = Path(f"./corpus_input")
## This is where the output csv files will be written
##### out_dir = data_dir
outputdirectory = Path(f"./corpus_output")

## Load Documents

In [10]:
! ls -l ./corpus_input

total 60
-rw-r--r-- 1 jupyter jupyter 60855 Mar  2 04:02 111-2.pdf


In [11]:
! ls -l ./corpus_input

total 60
-rw-r--r-- 1 jupyter jupyter 60855 Mar  2 04:02 111-2.pdf


In [12]:
loader = DirectoryLoader(inputdirectory, show_progress=True)
documents = loader.load()

splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,
    chunk_overlap=50,
    length_function=len,
    is_separator_regex=False,
)

pages = splitter.split_documents(documents)
print("Number of chunks = ", len(pages))
print(pages[0].page_content)


100%|██████████| 1/1 [00:07<00:00,  7.03s/it]

Number of chunks =  24
น้ำตาลในเลือดสูง น้ำตาลในเลือดสูงเสี่ยงเป็นโรคเบาหวาน

ปัจจุบันพบว่ามีคนจำนวนมากมีระดับน้ำตาลในเลือดสูงโดยไม่รู้ตัว ซึ่งคนเหล่านี้จะมีระดับ น้ำตาลในเลือดที่สูงกว่าปกติ แต่ไม่ถึงกับสูงในระดับที่เป็นโรคเบาหวาน ทำให้มีความเสี่ยงเพิ่ม ขึ้นในการเป็นโรคเบาหวานชนิดที่ 2 ซึ่งถ้าหากทราบและสามารถเปลี่ยนแปลงวิถีชีวิตได้ทัน ท่วงที ก็จะสามารถป้องกันและชะลอการเกิดโรคเบาหวานชนิดที่ 2 และปัญหาสุขภาพร้ายแร งอื่นๆ ได้


## Create a dataframe of all the chunks

In [13]:
import uuid
import pandas as pd
import numpy as np

def documents2Dataframe(documents) -> pd.DataFrame:
    rows = []
    for chunk in documents:
        row = {
            "text": chunk.page_content,
            **chunk.metadata,
            "chunk_id": uuid.uuid4().hex,
        }
        rows = rows + [row]

    df = pd.DataFrame(rows)
    return df

In [14]:
#from helpers.df_helpers import documents2Dataframe
df = documents2Dataframe(pages)
print(df.shape)
df.head()

(24, 3)


,text,source,chunk_id
0,น้ำตาลในเลือดสูง น้ำตาลในเลือดสูงเสี่ยงเป็นโรค...,corpus_input/111-2.pdf,53bc5a5661f94dcaaaa08abacf95ed5c
1,ผู้ที่มีภาวะน้ำตาลในเลือดสูงและไม่ได้รับการรัก...,corpus_input/111-2.pdf,c1a56beede0f454c917fa270fdadacef
2,ค่าปกติจะอยู่ระหว่าง 70-100 ระดับน้ำตาลในเลือด...,corpus_input/111-2.pdf,7aae8549ddaf4e1e8514a33e0636189c
3,อินซูลินเป็นฮอร์โมนในร่างกายของเราที่สร้างโดยต...,corpus_input/111-2.pdf,86626a7b674d473d91358c2c5207b23b
4,ปัจจัยเสี่ยงที่สำคัญสำหรับภาวะน้ำตาลในเลือดสูง...,corpus_input/111-2.pdf,ee59c94ece8a4ebb9cd49b1647de53ce


In [15]:
df

,text,source,chunk_id
0,น้ำตาลในเลือดสูง น้ำตาลในเลือดสูงเสี่ยงเป็นโรค...,corpus_input/111-2.pdf,53bc5a5661f94dcaaaa08abacf95ed5c
1,ผู้ที่มีภาวะน้ำตาลในเลือดสูงและไม่ได้รับการรัก...,corpus_input/111-2.pdf,c1a56beede0f454c917fa270fdadacef
2,ค่าปกติจะอยู่ระหว่าง 70-100 ระดับน้ำตาลในเลือด...,corpus_input/111-2.pdf,7aae8549ddaf4e1e8514a33e0636189c
3,อินซูลินเป็นฮอร์โมนในร่างกายของเราที่สร้างโดยต...,corpus_input/111-2.pdf,86626a7b674d473d91358c2c5207b23b
4,ปัจจัยเสี่ยงที่สำคัญสำหรับภาวะน้ำตาลในเลือดสูง...,corpus_input/111-2.pdf,ee59c94ece8a4ebb9cd49b1647de53ce
5,ผู้ที่มีน้ำตาลในเลือดสูงนานๆ อาจพัฒนาไปเป็นโรค...,corpus_input/111-2.pdf,5a1a66602fbb4db38334ef1c423c9694
6,เมื่อเรารับประทานอาหารเข้าไป ร่างกายจะสลายอาหา...,corpus_input/111-2.pdf,fac57044fbe74ddfaf4d357e4826c1d4
7,โรคเบาหวาน เกิดจากร่างกายของเราผลิตอินซูลินไม่...,corpus_input/111-2.pdf,f8a81f0ea8024b4a85fa19ca01760bc7
8,สัญญาณและอาการของโรคเบาหวานชนิดที่ 2 มักเกิดขึ...,corpus_input/111-2.pdf,fd8a06f2b52540ed907aef4c66b29687
9,โรคเบาหวานชนิดที่ 1 เกิดจากร่างกายของเราไม่สาม...,corpus_input/111-2.pdf,cc93bde4ca6a4ea38c09ef13ca3ee523


## Extract Concepts

In [16]:
import torch
from accelerate import Accelerator

accelerator = Accelerator()

In [17]:
from transformers import AutoModelForCausalLM, AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("Aekanun/openthaigpt-MedChatModel")
model = AutoModelForCausalLM.from_pretrained("Aekanun/openthaigpt-MedChatModel")

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

/opt/conda/lib/python3.10/site-packages/bitsandbytes/cextension.py:34: UserWarning: The installed version of bitsandbytes was compiled without GPU support. 8-bit optimizers, 8-bit multiplication, and GPU quantization are unavailable.
  warn("The installed version of bitsandbytes was compiled without GPU support. "


/opt/conda/lib/python3.10/site-packages/bitsandbytes/libbitsandbytes_cpu.so: undefined symbol: cadam32bit_grad_fp32


In [18]:
# ย้าย model ไปยัง device
##### model = model.to(accelerator.device)

In [19]:
## This function uses the helpers/prompt function to extract concepts from text
## from helpers.df_helpers import df2Graph
## from helpers.df_helpers import graph2Df

In [20]:
import openai
import sys
import json
import numpy as np
from yachalk import chalk
# Assuming ollama.client is a valid import in your environment
##### import ollama.client as client

sys.path.append("..")

def graphPrompt(input: str, metadata={}, model=model):

    print("LLMs (graphPrmpt) get input ===>", input)

    SYS_PROMPT = (
    "You are a network graph maker who extracts terms and their relations from a given context. "
    "You are provided with a context chunk (delimited by ```). Your task is to extract the ontology "
    "of terms mentioned in the given context, and keep original language."
    )

    ASSIST_PROMPT = (
        "These terms should represent the key concepts as per the context. \n"
        "Thought 1: While traversing through each sentence, think about the key terms mentioned in it.\n"
            "\tTerms may include object, entity, location, organization, person, \n"
            "\tcondition, acronym, documents, service, concept, etc.\n"
            "\tTerms should be as atomistic as possible\n\n"
        "Thought 2: Think about how these terms can have one on one relation with other terms.\n"
            "\tTerms that are mentioned in the same sentence or the same paragraph are typically related to each other.\n"
            "\tTerms can be related to many other terms\n\n"
        "Thought 3: Find out the relation between each such related pair of terms. \n\n"
        "Format your output as a list of json. Each element of the list contains a pair of terms "
        "and the relation between them, like the following: \n"
        "[\n"
        "   {\n"
        '       "node_1": "A concept from extracted ontology",\n'
        '       "node_2": "A related concept from extracted ontology",\n'
        '       "edge": "relationship between the two concepts, node_1 and node_2 in one or two sentences"\n'
        "   }, {...}\n"
        "]"
    )

    #custom_prompt = "การรักษาโรคโลหิตจางทาลัสซีเมีย ส่วนใหญ่เป็นการรักษาแบบประคับประคองโดยการให้เลือด ซึ่งมี ๒ แบบ คือ แบบที่ ๑ เป็นการให้เลือดแบบประคับประคองเมื่อผู้ป่วยซีดมาก เพื่อเพิ่มระดับเฮโมโกลบินขึ้นให้สูงพอที่ผู้ป่วยจะหายจากอาการอ่อนเพลียหรือเหนื่อยจากการขาดออกซิเจน โดยให้ตามความจำเป็นแบบที่ ๒ เป็นการให้เลือดจนผู้ป่วยมีระดับเฮโมโกลบินใกล้เคียงเด็กปกติ ซึ่งส่วนมากจะให้แก่ผู้ป่วยชนิดรุนแรงที่เริ่มมีอาการตั้งแต่อายุน้อยๆ การให้เลือดอย่างสม่ำเสมอจะยับยั้งการเปลี่ยนแปลงของลักษณะกระดูกใบหน้า ป้องกันไม่ให้ตับและม้ามโต ไม่เหนื่อย และมีการเจริญเติบโตใกล้เคียงเด็กปกติทั้งความสูงและน้ำหนักตัวแต่ข้อเสียของการให้เลือดวิธีนี้คือ ผู้ป่วยต้องมารับเลือดอย่างสม่ำเสมอและจะมีปัญหาที่สำคัญแทรกซ้อนตามมา คือภาวะธาตุเหล็กเกิน เพราะการให้เลือดทุก ๑ มิลลิลิตรของเม็ดเลือดแดง และมีธาตุเหล็กประมาณ ๑ มิลลิกรัม ธาตุเหล็กที่ได้รับนี้จะตกค้างอยู่ในร่างกายและไปสะสมในอวัยวะต่างๆ เช่น หัวใจ ตับ และตับอ่อน"
    custom_prompt = input


    prompt = f'''
    ### Instruction:
    {SYS_PROMPT}
    {ASSIST_PROMPT}

    ### context: {custom_prompt}
    ### Response:
    '''


    # Process the response from the model
    try:
        # Encode the prompt to input IDs and attention mask
        # Encode the prompt to input IDs and attention mask
        inputs = tokenizer(prompt, return_tensors='pt', padding=True, truncation=True, max_length=1000)

        # Generate output with explicit attention mask usage and setting pad_token_id if necessary
        # Note: For many models, the pad_token_id is automatically set to the model's default.
        # If you need to set it explicitly, you can do so in the model's config or directly as an argument in generate (if supported).
        outputs = model.generate(**inputs, max_length=1500, pad_token_id=tokenizer.eos_token_id)

        # Decode the generated ids to text and print
        generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)

        # ตัดเอาเฉพาะส่วน "Response" (สมมติว่าคุณทราบจุดเริ่มต้นของ "Response")
        response_start = generated_text.find("### Response:") + len("### Response:")
        response_text = generated_text[response_start:].strip()

        print ("RESPONSE TEXT ===========================>", response_text)

        # แสดงผลลัพธ์
        ##### return (response_text)
        import json
        import re
        # ใช้ regular expression สำหรับการหา matches ของ dictionaries ภายในข้อความ
        matches = re.findall(r"\{\n\s+\"node_1\": \"(.*?)\",\n\s+\"node_2\": \"(.*?)\",\n\s+\"edge\": \"(.*?)\"\n\s+\}", response_text)

        # สร้าง list ของ string โดยแต่ละอันเป็นการแสดงข้อมูลของ dictionary
        data_strings = [f'{{"node_1": "{match[0]}", "node_2": "{match[1]}", "edge": "{match[2]}"}}' for match in matches]

        # รวม list ของ string เป็น single string โดยใช้ comma และ newline เพื่อแยกแต่ละอัน
        data_text = "[\n" + ",\n".join(data_strings) + "\n]"

        print(data_text)
        result = json.loads(data_text)
        
        print ("JSON format ===========================>", result)
        
        
        result = [dict(item, **metadata) for item in result]

    except Exception as e:
        print(f"An error occurred: {e}")
        return []

    return result


In [21]:
import uuid
import pandas as pd
import numpy as np
#from .prompts import extractConcepts
#from .prompts import graphPrompt

def df2Graph(dataframe: pd.DataFrame, model) -> list:
    # dataframe.reset_index(inplace=True)
    results = dataframe.apply(
        lambda row: graphPrompt(row.text, {"chunk_id": row.chunk_id}, model), axis=1
    )
    # invalid json results in NaN
    results = results.dropna()
    results = results.reset_index(drop=True)

    ## Flatten the list of lists to one single list of entities.
    concept_list = np.concatenate(results).ravel().tolist()
    return concept_list


In [22]:
def graph2Df(nodes_list) -> pd.DataFrame:
    ## Remove all NaN entities
    graph_dataframe = pd.DataFrame(nodes_list).replace(" ", np.nan)
    graph_dataframe = graph_dataframe.dropna(subset=["node_1", "node_2"])
    graph_dataframe["node_1"] = graph_dataframe["node_1"].apply(lambda x: x.lower())
    graph_dataframe["node_2"] = graph_dataframe["node_2"].apply(lambda x: x.lower())

    return graph_dataframe

In [23]:
#model

In [24]:
##### concepts_list = df2Graph(df, model)

In [25]:
##### concepts_list

In [26]:
## To regenerate the graph with LLM, set this to True
regenerate = True

if regenerate:
    concepts_list = df2Graph(df, model)
    dfg1 = graph2Df(concepts_list)
    if not os.path.exists(outputdirectory):
        os.makedirs(outputdirectory)

    dfg1.to_csv(outputdirectory/"graph.csv", sep="|", index=False)
    df.to_csv(outputdirectory/"chunks.csv", sep="|", index=False)
else:
    dfg1 = pd.read_csv(outputdirectory/"graph.csv", sep="|")

LLMs (graphPrmpt) get input ===> น้ำตาลในเลือดสูง น้ำตาลในเลือดสูงเสี่ยงเป็นโรคเบาหวาน

ปัจจุบันพบว่ามีคนจำนวนมากมีระดับน้ำตาลในเลือดสูงโดยไม่รู้ตัว ซึ่งคนเหล่านี้จะมีระดับ น้ำตาลในเลือดที่สูงกว่าปกติ แต่ไม่ถึงกับสูงในระดับที่เป็นโรคเบาหวาน ทำให้มีความเสี่ยงเพิ่ม ขึ้นในการเป็นโรคเบาหวานชนิดที่ 2 ซึ่งถ้าหากทราบและสามารถเปลี่ยนแปลงวิถีชีวิตได้ทัน ท่วงที ก็จะสามารถป้องกันและชะลอการเกิดโรคเบาหวานชนิดที่ 2 และปัญหาสุขภาพร้ายแร งอื่นๆ ได้
RESPONSE TEXT ===========================> [
       {
         "node_1": "น้ำตาลในเลือดสูง",
         "node_2": "ความเสี่ยงเป็นโรคเบาหวาน",
         "edge": "ความสัมพันธ์ระหว่างน้ำตาลในเลือดสูงกับความเสี่ยงเป็นโรคเบาหวาน"
       },
       {
         "node_1": "น้ำตาลในเลือดสูง",
         "node_2": "โรคเบาหวาน",
         "edge": "ความสัมพันธ์ระหว่างน้ำตาลในเลือดสูงกับโรคเบาหวาน"
       },
       {
         "node_1": "น้ำตาลในเลือดสูง",
         "node_2": "โรคเบาหวานชนิดที่ 2",
         "edge": "ความสัมพันธ์ระหว่างน้ำตาลในเลือดสูงกับโรคเบาหวานชนิดที่ 2"
     

In [27]:
dfg1.replace("", np.nan, inplace=True)
dfg1.dropna(subset=["node_1", "node_2", 'edge'], inplace=True)
dfg1['count'] = 4

In [28]:
concepts_list

[{'node_1': 'น้ำตาลในเลือดสูง',
  'node_2': 'ความเสี่ยงเป็นโรคเบาหวาน',
  'edge': 'ความสัมพันธ์ระหว่างน้ำตาลในเลือดสูงกับความเสี่ยงเป็นโรคเบาหวาน',
  'chunk_id': '53bc5a5661f94dcaaaa08abacf95ed5c'},
 {'node_1': 'น้ำตาลในเลือดสูง',
  'node_2': 'โรคเบาหวาน',
  'edge': 'ความสัมพันธ์ระหว่างน้ำตาลในเลือดสูงกับโรคเบาหวาน',
  'chunk_id': '53bc5a5661f94dcaaaa08abacf95ed5c'},
 {'node_1': 'น้ำตาลในเลือดสูง',
  'node_2': 'โรคเบาหวานชนิดที่ 2',
  'edge': 'ความสัมพันธ์ระหว่างน้ำตาลในเลือดสูงกับโรคเบาหวานชนิดที่ 2',
  'chunk_id': '53bc5a5661f94dcaaaa08abacf95ed5c'},
 {'node_1': 'น้ำตาลในเลือดสูง',
  'node_2': 'สุขภาพ',
  'edge': 'ความสัมพันธ์ระหว่างน้ำตาลในเลือดสูงกับสุขภาพ',
  'chunk_id': '53bc5a5661f94dcaaaa08abacf95ed5c'},
 {'node_1': 'A concept from the context',
  'node_2': 'A related concept from the context',
  'edge': 'relationship between the two concepts, node_1 and node_2 in one or two sentences',
  'chunk_id': '53bc5a5661f94dcaaaa08abacf95ed5c'},
 {'node_1': 'น้ำตาลในเลือดสูง',
  'node_2

In [29]:
dfg1

,node_1,node_2,edge,chunk_id,count
0,น้ำตาลในเลือดสูง,ความเสี่ยงเป็นโรคเบาหวาน,ความสัมพันธ์ระหว่างน้ำตาลในเลือดสูงกับความเสี่...,53bc5a5661f94dcaaaa08abacf95ed5c,4
1,น้ำตาลในเลือดสูง,โรคเบาหวาน,ความสัมพันธ์ระหว่างน้ำตาลในเลือดสูงกับโรคเบาหวาน,53bc5a5661f94dcaaaa08abacf95ed5c,4
2,น้ำตาลในเลือดสูง,โรคเบาหวานชนิดที่ 2,ความสัมพันธ์ระหว่างน้ำตาลในเลือดสูงกับโรคเบาหว...,53bc5a5661f94dcaaaa08abacf95ed5c,4
3,น้ำตาลในเลือดสูง,สุขภาพ,ความสัมพันธ์ระหว่างน้ำตาลในเลือดสูงกับสุขภาพ,53bc5a5661f94dcaaaa08abacf95ed5c,4
4,a concept from the context,a related concept from the context,"relationship between the two concepts, node_1 ...",53bc5a5661f94dcaaaa08abacf95ed5c,4
...,...,...,...,...,...
290,การเดิน,การว่ายน้ำ,การออกกำลัง,9739439c611a4b2e9b1a83e40735009e,4
291,การเดิน,การว่ายน้ำ,การออกกำลัง,9739439c611a4b2e9b1a83e40735009e,4
292,การเดิน,การว่ายน้ำ,การออกกำลัง,9739439c611a4b2e9b1a83e40735009e,4
293,การเดิน,การว่ายน้ำ,การออกกำลัง,9739439c611a4b2e9b1a83e40735009e,4


## Calculating contextual proximity

In [41]:
def contextual_proximity(df: pd.DataFrame) -> pd.DataFrame:
    ## Melt the dataframe into a list of nodes
    dfg_long = pd.melt(
        df, id_vars=["chunk_id"], value_vars=["node_1", "node_2"], value_name="node"
    )
    dfg_long.drop(columns=["variable"], inplace=True)
    # Self join with chunk id as the key will create a link between terms occuring in the same text chunk.
    dfg_wide = pd.merge(dfg_long, dfg_long, on="chunk_id", suffixes=("_1", "_2"))
    # drop self loops
    self_loops_drop = dfg_wide[dfg_wide["node_1"] == dfg_wide["node_2"]].index
    dfg2 = dfg_wide.drop(index=self_loops_drop).reset_index(drop=True)
    ## Group and count edges.
    dfg2 = (
        dfg2.groupby(["node_1", "node_2"])
        .agg({"chunk_id": [",".join, "count"]})
        .reset_index()
    )
    dfg2.columns = ["node_1", "node_2", "chunk_id", "count"]
    dfg2.replace("", np.nan, inplace=True)
    dfg2.dropna(subset=["node_1", "node_2"], inplace=True)
    # Drop edges with 1 count
    dfg2 = dfg2[dfg2["count"] != 1]
    dfg2["edge"] = "contextual proximity"
    return dfg2


dfg2 = contextual_proximity(dfg1)
dfg2.tail()

,node_1,node_2,chunk_id,count,edge
355,โรคเบาหวานชนิดที่ 2,สุขภาพ,"53bc5a5661f94dcaaaa08abacf95ed5c,53bc5a5661f94...",4,contextual proximity
356,โรคเบาหวานชนิดที่ 2,อาการของโรคเบาหวาน,"f8a81f0ea8024b4a85fa19ca01760bc7,f8a81f0ea8024...",4,contextual proximity
357,โรคเบาหวานชนิดที่ 2,เบาหวาน,"f8a81f0ea8024b4a85fa19ca01760bc7,f8a81f0ea8024...",4,contextual proximity
358,โรคเบาหวานชนิดที่ 2,เบาหวานชนิดที่ 2,"6a8f1a790523498bb1e00f1c552d856f,6a8f1a7905234...",20,contextual proximity
359,โรคเบาหวานชนิดที่ 2,โรคเบาหวาน,"6a8f1a790523498bb1e00f1c552d856f,6a8f1a7905234...",436,contextual proximity


### Merge both the dataframes

In [42]:
dfg = pd.concat([dfg1, dfg2], axis=0)
dfg = (
    dfg.groupby(["node_1", "node_2"])
    .agg({"chunk_id": ",".join, "edge": ','.join, 'count': 'sum'})
    .reset_index()
)
dfg

,node_1,node_2,chunk_id,edge,count
0,a concept from extracted ontology,a related concept from extracted ontology,"5a1a66602fbb4db38334ef1c423c9694,5a1a66602fbb4...","relationship between the two concepts, node_1 ...",29
1,a concept from extracted ontology,การฉีดอินซูลิน,"eaaa922bab254c19bd823bf9caa5366e,eaaa922bab254...",contextual proximity,4
2,a concept from extracted ontology,การรักษา,"eaaa922bab254c19bd823bf9caa5366e,eaaa922bab254...",contextual proximity,4
3,a concept from extracted ontology,การรับประทานยา,"eaaa922bab254c19bd823bf9caa5366e,eaaa922bab254...",contextual proximity,4
4,a concept from extracted ontology,การลดน้ำหนัก,"eaaa922bab254c19bd823bf9caa5366e,eaaa922bab254...",contextual proximity,4
...,...,...,...,...,...
265,โรคเบาหวานชนิดที่ 2,สุขภาพ,"53bc5a5661f94dcaaaa08abacf95ed5c,53bc5a5661f94...",contextual proximity,4
266,โรคเบาหวานชนิดที่ 2,อาการของโรคเบาหวาน,"f8a81f0ea8024b4a85fa19ca01760bc7,f8a81f0ea8024...",contextual proximity,4
267,โรคเบาหวานชนิดที่ 2,เบาหวาน,"f8a81f0ea8024b4a85fa19ca01760bc7,f8a81f0ea8024...",contextual proximity,4
268,โรคเบาหวานชนิดที่ 2,เบาหวานชนิดที่ 2,"6a8f1a790523498bb1e00f1c552d856f,6a8f1a7905234...",contextual proximity,20


## Calculate the NetworkX Graph

In [43]:
nodes = pd.concat([dfg['node_1'], dfg['node_2']], axis=0).unique()
nodes.shape

(57,)

In [44]:
import networkx as nx
G = nx.Graph()

## Add nodes to the graph
for node in nodes:
    G.add_node(
        str(node)
    )

## Add edges to the graph
for index, row in dfg.iterrows():
    G.add_edge(
        str(row["node_1"]),
        str(row["node_2"]),
        title=row["edge"],
        weight=row['count']/4
    )

### Calculate communities for coloring the nodes

In [45]:
communities_generator = nx.community.girvan_newman(G)
top_level_communities = next(communities_generator)
next_level_communities = next(communities_generator)
communities = sorted(map(sorted, next_level_communities))
print("Number of Communities = ", len(communities))
print(communities)

Number of Communities =  7
[['a concept from extracted ontology', 'a related concept from extracted ontology', 'การฉีดอินซูลิน', 'การรักษา', 'การรับประทานยา', 'การลดน้ำหนัก', 'การออกกำลังกาย', 'ตาบอด', 'น้ำตาลในเลือดสูง', 'ผิวหนัง', 'อาการของโรคเบาหวาน', 'เชื้อรา', 'เบาหวาน', 'เบาหวานชนิดที่ 2', 'แบคทีเรีย', 'โรคหัวใจและหลอดเลือด', 'โรคเบาหวาน', 'โรคเบาหวานชนิดที่ 2'], ['a concept from the context', 'a related concept from the context', 'การควบคุมน้ำหนัก', 'การควบคุมระดับน้ำตาลในเลือดที่ไม่ดี', 'ความเสี่ยงเป็นโรคเบาหวาน', 'ตับอ่อน', 'น้ำตาล', 'น้ำตาลในเลือด', 'ภาวะสมองเสื่อม', 'ภาวะแทรกซ้อนของโรคเบาหวาน', 'สุขภาพ', 'สูง', 'อาหารเพื่อสุขภาพ', 'อินซูลิน', 'ฮอร์โมน', 'เซลล์', 'โรคสมองเสื่อม', 'โรคเบาหวานขณะตั้งครรภ์', 'โรคเบาหวานชนิดที่ 1'], ['a concept from the given context', 'a related concept from the given context'], ['การปรับเปลี่ยนวิถีชีวิต', 'การวินิจฉัยภาวะน้ำตาลในเลือดสูง'], ['การปั่นจักรยาน', 'การวิ่ง', 'การว่ายน้ำ', 'การเดิน'], ['การป้องกันโรคเบาหวาน', 'การป้องกันไม่ให้ขาดวิตา

### Create a dataframe for community colors

In [46]:
import seaborn as sns
palette = "hls"

## Now add these colors to communities and make another dataframe
def colors2Community(communities) -> pd.DataFrame:
    ## Define a color palette
    p = sns.color_palette(palette, len(communities)).as_hex()
    random.shuffle(p)
    rows = []
    group = 0
    for community in communities:
        color = p.pop()
        group += 1
        for node in community:
            rows += [{"node": node, "color": color, "group": group}]
    df_colors = pd.DataFrame(rows)
    return df_colors


colors = colors2Community(communities)
colors

,node,color,group
0,a concept from extracted ontology,#db57c0,1
1,a related concept from extracted ontology,#db57c0,1
2,การฉีดอินซูลิน,#db57c0,1
3,การรักษา,#db57c0,1
4,การรับประทานยา,#db57c0,1
5,การลดน้ำหนัก,#db57c0,1
6,การออกกำลังกาย,#db57c0,1
7,ตาบอด,#db57c0,1
8,น้ำตาลในเลือดสูง,#db57c0,1
9,ผิวหนัง,#db57c0,1


### Add colors to the graph

In [47]:
for index, row in colors.iterrows():
    G.nodes[row['node']]['group'] = row['group']
    G.nodes[row['node']]['color'] = row['color']
    G.nodes[row['node']]['size'] = G.degree[row['node']]

In [48]:
from pyvis.network import Network

graph_output_directory = "./docs/index.html"

net = Network(
    notebook=False,
    # bgcolor="#1a1a1a",
    cdn_resources="remote",
    height="900px",
    width="100%",
    select_menu=True,
    # font_color="#cccccc",
    filter_menu=False,
)

net.from_nx(G)
# net.repulsion(node_distance=150, spring_length=400)
net.force_atlas_2based(central_gravity=0.015, gravity=-31)
# net.barnes_hut(gravity=-18100, central_gravity=5.05, spring_length=380)
net.show_buttons(filter_=["physics"])

net.show(graph_output_directory, notebook=False)

./docs/index.html


In [49]:
! ls -l ./docs

total 84
-rw-r--r-- 1 jupyter jupyter 82066 Mar  2 11:31 index.html


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [50]:
! ls -l corpus_output

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


total 80
-rw-r--r-- 1 jupyter jupyter 24955 Mar  2 11:21 chunks.csv
-rw-r--r-- 1 jupyter jupyter 50440 Mar  2 11:21 graph.csv


In [51]:
! head -10 corpus_output/graph.csv

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


node_1|node_2|edge|chunk_id
น้ำตาลในเลือดสูง|ความเสี่ยงเป็นโรคเบาหวาน|ความสัมพันธ์ระหว่างน้ำตาลในเลือดสูงกับความเสี่ยงเป็นโรคเบาหวาน|53bc5a5661f94dcaaaa08abacf95ed5c
น้ำตาลในเลือดสูง|โรคเบาหวาน|ความสัมพันธ์ระหว่างน้ำตาลในเลือดสูงกับโรคเบาหวาน|53bc5a5661f94dcaaaa08abacf95ed5c
น้ำตาลในเลือดสูง|โรคเบาหวานชนิดที่ 2|ความสัมพันธ์ระหว่างน้ำตาลในเลือดสูงกับโรคเบาหวานชนิดที่ 2|53bc5a5661f94dcaaaa08abacf95ed5c
น้ำตาลในเลือดสูง|สุขภาพ|ความสัมพันธ์ระหว่างน้ำตาลในเลือดสูงกับสุขภาพ|53bc5a5661f94dcaaaa08abacf95ed5c
a concept from the context|a related concept from the context|relationship between the two concepts, node_1 and node_2 in one or two sentences|53bc5a5661f94dcaaaa08abacf95ed5c
น้ำตาลในเลือดสูง|ความเสี่ยงเป็นโรคเบาหวาน|ความสัมพันธ์ระหว่างน้ำตาลในเลือดสูงกับความเสี่ยงเป็นโรคเบาหวาน|53bc5a5661f94dcaaaa08abacf95ed5c
น้ำตาลในเลือดสูง|โรคเบาหวาน|ความสัมพันธ์ระหว่างน้ำตาลในเลือดสูงกับโรคเบาหวาน|53bc5a5661f94dcaaaa08abacf95ed5c
น้ำตาลในเลือดสูง|โรคเบาหวานชนิดที่ 2|ความสัมพันธ์ระหว่างน้ำตาลในเลือดส